# Dinh et al. (2019)

## A data-driven approach to predicting diabetes and cardiovascular disease with machine learning

URL: https://bmcmedinformdecismak.biomedcentral.com/articles/10.1186/s12911-019-0918-5


## Brief Summary

Dinh et al. (2019) uses different ML models (logistic regression, support vector machines, random forest, and gradient boosting) on NHANES dataset to predict i) Diabetes and ii) Cardiovascular disease ("CVD").

**Goal**: Identification mechanism for patients at risk of diabetes and cardiovascular diseases and key contributors to diabetes .

**Results**:

Best scores:

- CVB prediction based on 131 NHANES variables achieved an AU-ROC score of 83.9% .
- Diabetes prediction based on 123 NHANES variables achieved an AU-ROC score of 95.7% .
- Pre-diabetic prediction based on 123 NHANES variables achieved an AU-ROC score of 84.4% .
- Top 5 predictors in diabetes patients were 1) `waist size`, 2) `age`, 3) `self-reported weight`, 4) `leg length`, 5) `sodium intake`.



This notebook replicates the results of the paper. The structure follows the following steps: 

1. NHANES data 
2. Pre-processing of the data
3. Transformation of the data
4. Train/Test Split 
5. CV 10-fold
6. Training monitoring using MLflow
7. Get metric results (AUC)


The structure of the analysis emulates the Figure 1 from the paper: 

![Fig 1 from Dinh et al. 2019](https://raw.githubusercontent.com/pipegalera/ml_diabetes/main/images/dinh_2019_Fig1.png)


In [1]:
library(arrow)
library(dplyr)
library(readxl)

Some features are not enabled in this build of Arrow. Run `arrow_info()` for more information.

The repository you retrieved Arrow from did not include all of Arrow's features.
You can install a fully-featured version by running:
`install.packages('arrow', repos = 'https://apache.r-universe.dev')`.


Attaching package: 'arrow'


The following object is masked from 'package:utils':

    timestamp



Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




## 1. HNANES data

URL: https://www.cdc.gov/nchs/index.htm


## Target

From the paper, the definitions are clear: 

![Dinh et al.(2019), Table 4](https://raw.githubusercontent.com/pipegalera/ml_diabetes/main/images/dinh_2019_Table4.png)

- Case I: Diabetes.

    - Glucose >= 126 mg/dL. OR;
    - "Yes" to the question "Have you ever been told by a doctor that you have diabetes?"

- Case II: Undiagnosed Diabetes. 

    - Glucose >= 126 mg/dL. AND;
    - "No" to the question "Have you ever been told by a doctor that you have diabetes?"

- Cardio: Cardiovascular disease.

    - "Yes" to any of the the questions "Have you ever been told by a doctor that you had congestive heart failure, coronary heart disease, a heart attack, or a stroke?"

The paper also defined and test for the target: 

- Pre diabetes

    - Glucose 125 >= 100 mg/dL

## Covariates

The paper did not say what variables they use from NHANES. I emailed the author in the correspondence section of the paper to try to get the list of variables they used, but no answer from him yet.

Given that NHANES have more than 3000 variables, I cannot just randomly take the variables I believe are important. 

For now, I will consider the variables taken from [Figure 5](https://raw.githubusercontent.com/pipegalera/ml_diabetes/main/images/dinh_2019_Fig5.png) and [Figure 6](https://raw.githubusercontent.com/pipegalera/ml_diabetes/main/images/dinh_2019_Fig6.png) of the paper. I compiled them by hand in an Excel file using NHANES search tool for variables:



In [8]:
DATA_PATH <- "/Users/pipegalera/dev/ml_diabetes/data/NHANES/raw_data/"
dinh_2019_vars <- read_excel(paste0(DATA_PATH, "dinh_2019_variables_doc.xlsx"))

head(dinh_2019_vars[, c("Variable Name", "NHANES Name")], n=15)


Variable Name,NHANES Name
<chr>,<chr>
Age,RIDAGEYR
Alcohol consumption,ALQ130
Alcohol intake,DRXTALCO
"Alcohol intake, First Day",DR1TALCO
"Alcohol intake, Second Day",DR2TALCO
Arm circumference,BMXARMC
Arm length,BMXARML
Blood osmolality,LBXSOSSI
Blood relatives have diabetes,MCQ250A


For the complete list (n=62), check the file `dinh_2019_variables_doc.xlsx` under NHANES data folder.

NHANES data is made by multiple files (see `NHANES` unde data folder) that have to be compiled together. The data was downloaded automatically via script, all the files converted from SAS to parquet, and the files were stacked and merged based on the individual index ("SEQN"). For more details please check the `nhanes_data_backfill` notebook. 

Plese notice that no transformation are made to the covariates, the files were only arranged and stacked together. 

In [15]:
df <- read_parquet(paste0(DATA_PATH, "dinh_raw_data.parquet"))
head(df)

SEQN,YEAR,RIDAGEYR,ALQ130,DRXTALCO,DR1TALCO,DR2TALCO,BMXARMC,BMXARML,LBXSOSSI,...,BPXSY1,BPXSY4,BPXSY2,BPXSY3,LBDTCSI,LBDSTRSI,BMXWAIST,BMXWT,LBXWBCSI,LBXSASSI
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1999-2000,2,NA,5.397605e-79,NA,NA,15.2,18.6,NA,...,NA,NA,NA,NA,NA,NA,45.7,12.5,NA,NA
2,1999-2000,77,1,5.397605e-79,NA,NA,29.8,38.2,288,...,106,NA,98,98,5.56,1.298,98.0,75.4,7.6,19
3,1999-2000,10,NA,5.397605e-79,NA,NA,19.7,25.5,NA,...,110,NA,104,112,3.34,NA,64.7,32.9,7.5,NA
4,1999-2000,1,NA,5.397605e-79,NA,NA,16.4,20.4,NA,...,NA,NA,NA,NA,NA,NA,NA,13.3,8.8,NA
5,1999-2000,49,3,3.456000e+01,NA,NA,35.8,39.7,276,...,122,NA,122,122,7.21,3.850,99.9,92.5,5.9,22
6,1999-2000,19,NA,5.397605e-79,NA,NA,26.0,34.5,277,...,116,NA,116,112,3.96,0.553,81.6,59.2,9.6,20


In [16]:
tail(df)

SEQN,YEAR,RIDAGEYR,ALQ130,DRXTALCO,DR1TALCO,DR2TALCO,BMXARMC,BMXARML,LBXSOSSI,...,BPXSY1,BPXSY4,BPXSY2,BPXSY3,LBDTCSI,LBDSTRSI,BMXWAIST,BMXWT,LBXWBCSI,LBXSASSI
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
102951,2017-2018,4,NA,NA,5.397605e-79,5.397605e-79,20.9,24.0,NA,...,NA,NA,NA,NA,NA,NA,62.2,23.8,8.5,NA
102952,2017-2018,70,NA,NA,5.397605e-79,5.397605e-79,25.1,32.6,288,...,136,NA,142,140,3.08,1.106,82.2,49.0,5.1,27
102953,2017-2018,42,12,NA,5.397605e-79,NA,40.6,36.6,289,...,124,NA,122,116,4.71,1.287,114.8,97.4,8.3,29
102954,2017-2018,41,NA,NA,5.397605e-79,5.397605e-79,26.8,35.2,272,...,116,NA,118,114,4.45,0.723,86.4,69.1,5.1,15
102955,2017-2018,14,NA,NA,5.397605e-79,5.397605e-79,44.5,35.0,274,...,114,NA,114,114,3.88,1.005,113.5,111.9,11.4,16
102956,2017-2018,38,2,NA,5.397605e-79,5.397605e-79,40.0,38.0,277,...,150,NA,146,148,4.22,3.263,122.0,111.5,9.0,27


In [17]:
nrow(df)


[1] 101316

# 2. Pre-processing

There are some fixes before the data is ready for analysis. For example, variables that are the same but are called diffrent in different NHANES years:

1. Intake variables went from 1 day in 1999 to 2001 to 2 days from 2003 on, therefore the variable has to be homogenized. Dinh et al. (2019) do not specify which examination records the authors, but my best guess is that they problably took the average of both days that the examination was performed. 

This situation happends with:

- Alcohol intake (`DRXTALCO`, `DR1TALCO`, `DR2TALCO`)
- Caffeine intake (`DRXTCAFF`, `DR1TCAFF`, `DR2TCAFF`)
- Calcium intake (`DRXTCALC`, `DR1TCALC`, `DR2TCALC`)
- Carbohydrate intake (`DRXTCARB`, `DR1TCARB`, `DR2TCARB`)
- Fiber intake (`DRXTFIBE`, `DR1TFIBE`, `DR2TFIBE`)
- Kcal intake (`DRXTKCAL`, `DR1TKCAL`, `DR2TKCAL`)
- Sodium intake (`DRDTSODI`, `DR1TSODI`, `DR2TSODI`)


In [21]:
unique(df$YEAR[!is.na(df$DRXTALCO)])


[1] "1999-2000" "2001-2002"

In [23]:
unique(df$YEAR[!is.na(df$DR1TALCO)])


[1] "2003-2004" "2005-2006" "2007-2008" "2009-2010" "2011-2012" "2013-2014"
[7] "2015-2016" "2017-2018"

In [31]:
df$test <- ifelse(is.na(df$DRXTALCO), (df$DR1TALCO + df$DR1TALCO)/2, df$DRXTALCO)

In [46]:
unique(df$DR1TALCO[!is.na(df$DR1TALCO)])

[1] 5.397605e-79 3.350000e+01 5.040000e+01 6.440000e+01 2.810000e+01
   [6] 6.050000e+01 1.290000e+01 1.410000e+01 1.674000e+02 5.580000e+01
  [11] 1.530000e+01 5.620000e+01 1.000000e-01 1.040000e+01 3.400000e+01
  [16] 1.050000e+01 5.870000e+01 3.270000e+01 1.211000e+02 7.500000e+00
  [21] 3.000000e-01 5.800000e+01 2.120000e+01 2.300000e+00 9.000000e-01
  [26] 6.080000e+01 1.404000e+02 1.120000e+01 1.870000e+01 1.400000e+01
  [31] 7.430000e+01 2.360000e+01 3.050000e+01 7.660000e+01 6.540000e+01
  [36] 1.200000e+00 2.870000e+01 7.700000e+00 7.020000e+01 6.400000e+00
  [41] 2.000000e-01 2.230000e+01 3.900000e+01 1.570000e+01 2.139000e+02
  [46] 1.685000e+02 2.450000e+01 1.836000e+02 1.328000e+02 8.420000e+01
  [51] 8.430000e+01 1.339000e+02 4.680000e+01 5.000000e-01 3.710000e+01
  [56] 6.700000e+01 8.930000e+01 2.106000e+02 3.740000e+01 2.050000e+01
  [61] 4.210000e+01 9.400000e+00 3.070000e+01 1.350000e+01 1.079000e+02
  [66] 7.000000e-01 1.116000e+02 1.640000e+01 1.230000e+01 7.000000e+00
  [71] 1.800000e+00 9.300000e+00 8.500000e+00 3.030000e+01 6.380000e+01
  [76] 1.305000e+02 1.840000e+01 6.550000e+01 2.100000e+00 6.000000e-01
  [81] 4.240000e+01 1.910000e+01 2.020000e+01 2.150000e+01 4.280000e+01
  [86] 1.123000e+02 7.810000e+01 3.390000e+01 2.190000e+01 2.000000e+00
  [91] 1.130000e+01 1.585000e+02 5.300000e+00 7.370000e+01 4.600000e+01
  [96] 5.570000e+01 1.700000e+00 2.950000e+01 3.206000e+02 1.114000e+02
 [101] 2.117000e+02 7.460000e+01 4.000000e-01 6.500000e+00 2.140000e+01
 [106] 7.490000e+01 4.030000e+01 3.020000e+01 1.860000e+01 1.100000e+00
 [111] 6.970000e+01 2.570000e+01 1.260000e+01 2.630000e+01 5.350000e+01
 [116] 1.536000e+02 1.264000e+02 9.830000e+01 8.340000e+01 1.160000e+01
 [121] 3.130000e+01 6.110000e+01 2.520000e+01 5.560000e+01 1.300000e+00
 [126] 2.560000e+01 1.004000e+02 1.412000e+02 5.590000e+01 4.600000e+00
 [131] 2.240000e+01 4.300000e+01 8.400000e+00 3.220000e+01 1.440000e+02
 [136] 5.160000e+01 1.770000e+01 9.800000e+00 9.820000e+01 4.100000e+01
 [141] 8.600000e+01 6.270000e+01 6.910000e+01 1.390000e+01 2.583000e+02
 [146] 4.140000e+01 1.931000e+02 3.560000e+01 1.021000e+02 1.250000e+01
 [151] 1.409000e+02 1.029000e+02 1.242000e+02 3.900000e+00 1.490000e+01
 [156] 2.100000e+01 2.678000e+02 1.008000e+02 9.330000e+01 3.390000e+01
 [161] 3.720000e+01 6.900000e+01 2.400000e+01 2.920000e+01 7.560000e+01
 [166] 1.670000e+01 1.117000e+02 1.880000e+01 3.680000e+01 1.600000e+00
 [171] 2.340000e+01 5.810000e+01 5.110000e+01 2.200000e+00 3.840000e+01
 [176] 8.000000e-01 1.694000e+02 5.000000e+00 1.990000e+01 1.320000e+01
 [181] 1.486000e+02 4.740000e+01 4.100000e+00 1.500000e+00 4.300000e+00
 [186] 3.960000e+01 1.300000e+00 3.370000e+01 4.460000e+01 2.580000e+01
 [191] 1.386000e+02 6.500000e+01 2.170000e+01 4.230000e+01 1.341000e+02
 [196] 1.460000e+01 1.453000e+02 3.750000e+01 2.830000e+01 5.800000e+00
 [201] 8.290000e+01 1.018000e+02 5.910000e+01 2.980000e+01 5.500000e+01
 [206] 5.610000e+01 7.270000e+01 1.480000e+01 3.090000e+01 1.170000e+01
 [211] 2.300000e+01 1.730000e+01 4.380000e+01 2.009000e+02 9.660000e+01
 [216] 7.290000e+01 7.960000e+01 2.900000e+01 5.340000e+01 8.800000e+00
 [221] 1.630000e+01 1.281000e+02 5.640000e+01 2.209000e+02 6.000000e+01
 [226] 1.104000e+02 1.213000e+02 2.655000e+02 1.800000e+00 1.147000e+02
 [231] 5.600000e+00 9.360000e+01 8.860000e+01 7.700000e+01 5.900000e+00
 [236] 1.613000e+02 7.900000e+01 5.720000e+01 3.240000e+01 1.070000e+01
 [241] 6.590000e+01 3.226000e+02 3.520000e+01 1.698000e+02 2.504000e+02
 [246] 4.200000e+01 6.780000e+01 9.000000e-01 3.530000e+01 2.320000e+01
 [251] 6.000000e+00 4.180000e+01 4.020000e+01 1.400000e+00 3.930000e+01
 [256] 2.510000e+01 3.200000e+00 5.240000e+01 3.100000e+00 1.075000e+02
 [261] 6.510000e+01 2.700000e+01 7.420000e+01 4.980000e+01 1.109000e+02
 [266] 2.900000e+00 6.790000e+01 4.880000e+01 1.807000e+02 5.840000e+01
 [271] 9.000000e+00 9.250000e+01 9.100000e+00 1.290000e+02 1.681000e+02
 [276] 4.290000e+01 6.420000e+01 5.320000e+

In [52]:
df |>
filter(DR1TALCO == 58.9)

SEQN,YEAR,RIDAGEYR,ALQ130,DRXTALCO,DR1TALCO,DR2TALCO,BMXARMC,BMXARML,LBXSOSSI,...,BPXSY4,BPXSY2,BPXSY3,LBDTCSI,LBDSTRSI,BMXWAIST,BMXWT,LBXWBCSI,LBXSASSI,test
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
80519,2013-2014,36,3,NA,58.9,5.397605e-79,33.1,33.3,270,...,NA,96,106,5.30,1.964,107.0,71.8,6.9,19,58.9
93076,2015-2016,35,2,NA,58.9,5.397605e-79,35.5,39.6,281,...,NA,116,116,4.97,0.305,86.3,81.9,5.6,23,58.9


In [43]:
df$SEQN <- as.integer(df$SEQN)
df$RIDAGEYR <- as.integer(df$RIDAGEYR)
df$RIAGENDR <- as.integer(df$RIAGENDR)


In [41]:
filtered_df <- df |> 
  filter(RHD143 == 2) |>  # Are you pregnant now? = "No"
  filter(RIDAGEYR >= 20) |>
  filter(YEAR != "2015-2016/")

> The data was further analyzed for missing values within the variables, and any with more than 50% of missing values were dropped from the dataset.

Since the pregnancy question (RHD143) was only for *Females only 20 YEARS - 44 YEARS*, the missing values are men or women out of that age range, so I will not delete them.

In [51]:
missing_values <- sapply(df, function(x) mean(is.na(x))) * 100
missing_values_df <- data.frame(missing_percentage = missing_values) |>
                        arrange(desc(missing_percentage))
print(missing_values_df)

         missing_percentage
RHD143            93.640156
BPXDI4            91.407964
BPXSY4            91.407964
LBDLDL            80.323043
LBDTRSI           80.009124
LBDGLUSI          79.577893
LBXGH             57.676349
BPXDI3            33.969499
BPXSY3            33.968413
BPXDI2            33.337316
BPXSY2            33.337316
BPXDI1            32.356455
BPXSY1            32.356455
BPXPLS            27.077404
BMXWAIST          16.254263
BMXBMI            13.325802
BMXHT             12.859812
BMXWT              5.930786
DIQ010             4.515435
SEQN               0.000000
RIDAGEYR           0.000000
RIAGENDR           0.000000
YEAR               0.000000


In [ ]:



seeing if there is any extreme value that might be due to bad input of the data. According to the paper:

> The preprocessing stage also converted any undecipherable values (errors in datatypes and standard formatting) from the database to null representations.

In [ ]:
boxplot(df[, c('col1', 'col2', 'colN')])


In [ ]:
normalize_columns <- function(master, columns) {
  for (column in columns) {
    master[, column] <- (master[, column] - min(master[, column])) / (max(master[, column]) - min(master[, column]))
  }
  return(master)
}
